In [1]:
# Load Data Cube Configuration
import datacube
import datacube.utils.rio
from datacube.utils.cog import write_cog

import matplotlib.pyplot as plt
from odc.ui import DcViewer
import numpy as np
from scipy import stats
import xarray as xr


# Enable importing of utilities.
import sys
import os

#import DE Africa script
sys.path.append("../Scripts")
from deafrica_datahandling import mostcommon_crs, load_ard
from deafrica_bandindices import calculate_indices
from deafrica_plotting import display_map, rgb

#This will speed up loading data
datacube.utils.rio.set_default_rio_config(aws='auto', cloud_defaults=True)

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


In [2]:
#introduce yourself and state your purpose
dc = datacube.Datacube(app="Urban_Agriculture")

In [3]:
#whats available?
products = dc.list_products()

display_columns = ["name",
                   "description",
                   "platform",
                   "instrument",
                   "crs",
                   "resolution"]

products[display_columns].sort_index()

,name,description,platform,instrument,crs,resolution
id,,,,,,
1,ls8_usgs_sr_scene,Landsat 8 USGS Collection 1 Higher Level SR sc...,LANDSAT_8,OLI_TIRS,NaN,NaN
2,ls7_usgs_sr_scene,Landsat 7 USGS Collection 1 Level2 Surface Ref...,LANDSAT_7,ETM,NaN,NaN
3,ls5_usgs_sr_scene,Landsat 5 USGS Collection 1 Level2 Surface Ref...,LANDSAT_5,TM,NaN,NaN
5,ls_usgs_wofs_scene,Historic Flood Mapping Water Observations from...,None,None,NaN,NaN
6,ls_usgs_fc_scene,Landsat Fractional Cover based on USGS Level 2...,None,None,NaN,NaN
11,ls_usgs_wofs_summary,Water Observations from Space Annual Statistics,None,None,ESRI:102022,"(-30, 30)"
12,srtm,1 second elevation model,SRTM,SIR,EPSG:4326,"(-0.00027777777778, 0.00027777777778)"
13,ga_ls8c_wofs_2_annual_summary,Water Observations from Space Annual Statistics,None,None,EPSG:6933,"(-30, 30)"
14,alos_palsar_mosaic,ALOS/PALSAR and ALOS-2/PALSAR-2 annual mosaic ...,ALOS/ALOS-2,PALSAR/PALSAR-2,EPSG:4326,"(-0.000222222222222, 0.000222222222222)"


In [4]:
#have a look at Sentinel-2B
product = ['s2_l2a']

measurements = dc.list_measurements()
measurements.loc[product]

name   dtype units nodata                       aliases  \
product measurement                                                           
s2_l2a  B01          B01  uint16     1      0    [band_01, coastal_aerosol]   
        B02          B02  uint16     1      0               [band_02, blue]   
        B03          B03  uint16     1      0              [band_03, green]   
        B04          B04  uint16     1      0                [band_04, red]   
        B05          B05  uint16     1      0         [band_05, red_edge_1]   
        B06          B06  uint16     1      0         [band_06, red_edge_2]   
        B07          B07  uint16     1      0         [band_07, red_edge_3]   
        B08          B08  uint16     1      0         [band_08, nir, nir_1]   
        B8A          B8A  uint16     1      0  [band_8a, nir_narrow, nir_2]   
        B09          B09  uint16     1      0       [band_09, water_vapour]   
        B11          B11  uint16     1      0    [band_11, swir_1, swir_16]   
        B12          B12  uint16     1      0    [band_12, swir_2, swir_22]   
        SCL          SCL   uint8     1      0                    [mask, qa]   
        AOT          AOT  uint16     1      0   [aerosol_optical_thickness]   
        WVP          WVP  uint16     1      0  [scene_average_water_vapour]   

                                                      flags_definition  
product measurement                                                     
s2_l2a  B01                                                        NaN  
        B02                                                        NaN  
        B03                                                        NaN  
        B04                                                        NaN  
        B05                                                        NaN  
        B06                                                        NaN  
        B07                                                        NaN  
        B08                                                        NaN  
        B8A                                                        NaN  
        B09                                                        NaN  
        B11                                                        NaN  
        B12                                                        NaN  
        SCL          {'qa': {'bits': [0, 1, 2, 3, 4, 5, 6, 7], 'val...  
        AOT                                                        NaN  
        WVP                                                        NaN

In [5]:
# Set the central latitude and longitude - Cape Town - Philippi  Horticultural Area
central_lat = 18.56
central_lon = -34.04

# Set the buffer to load around the central coordinates
buffer = 0.05

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

In [6]:
#its always a good idea to see whats available -- note the option to choose a sensor on the upper left
DcViewer(dc=dc, 
         time='2020',
         center=(central_lon, central_lat),
         zoom=12)

In [7]:
query = {
    'time': ('2017-01-01', '2020-03-03'),
    'x': study_area_lat,
    'y': study_area_lon,
    'output_crs': 'epsg:32734',
    'resolution':(-10,10),
}

In [14]:
#load data

bands = ['blue', 'green', 'red', 'nir', 'swir_1']
#bands = ['B02', 'B03', 'B04', 'B08', 'B11']

ds = dc.load(product='s2_l2a',
             measurements=bands,
             dask_chunks={},
             **query)

print(ds)

<xarray.Dataset>
Dimensions:      (time: 207, x: 951, y: 1132)
Coordinates:
  * time         (time) datetime64[ns] 2017-01-01T08:45:43 ... 2020-03-01T08:49:50
  * y            (y) float64 6.236e+06 6.236e+06 ... 6.225e+06 6.225e+06
  * x            (x) float64 2.7e+05 2.7e+05 2.7e+05 ... 2.795e+05 2.795e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 dask.array<chunksize=(1, 1132, 951), meta=np.ndarray>
    green        (time, y, x) uint16 dask.array<chunksize=(1, 1132, 951), meta=np.ndarray>
    red          (time, y, x) uint16 dask.array<chunksize=(1, 1132, 951), meta=np.ndarray>
    nir          (time, y, x) uint16 dask.array<chunksize=(1, 1132, 951), meta=np.ndarray>
    swir_1       (time, y, x) uint16 dask.array<chunksize=(1, 1132, 951), meta=np.ndarray>
Attributes:
    crs:           epsg:32734
    grid_mapping:  spatial_ref


In [12]:
#how many images are there?
print('There is/are:', len(ds.time), 'image/s available')

There is/are: 207 image/s available


In [13]:
rgb(ds, index=[0,1,2,3])

KeyError: 'red'

In [11]:
ds = load_ard(dc=dc,
              products=['s2_l2a'],
              measurements=bands,
              dask_chunks={},
              **query)

print(ds)

Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a


KeyError: 'scl'